In [1]:
import requests
import pandas as pd

In [2]:
r = requests.get('https://nbviewer.org/github/rogovich/Data/blob/master/docs/Plotly_HW/FairVote%20-%20Press%20Room.html')
r

<Response [200]>

In [3]:
range(r.text.find('<br><br>Rank&nbsp'), r.text.find('* Source: Washington Post'))

range(15527, 22237)

In [4]:
range(r.text.find('<br><br>Rank&nbsp;&nbsp;&nbsp; State&nbsp;&nbsp;&nbsp; Ad'), r.text.find('* Source: <a href='))

range(24787, 28731)

In [5]:
range(r.text.find('<br>Year&nbsp'), r.text.find(' 16<br><br>As the'))

range(30236, 34473)

In [6]:
table_1 = r.text[15527:22237]
table_2 = r.text[24787:28731]
table_3 = r.text[30236:34473]
tables = []
tables.append(table_1)
tables.append(table_2)
tables.append(table_3)

In [7]:
n = 0

for table in tables:
    table = table.replace('&nbsp', ';')
    table = ';'.join([letter for letter in table.split(';') if len(letter)>0])
    if n == 0:
        a = []
        for i in table.split('<br>')[2:-2]:
            a.append(i.split(';'))
        n += n+1
    elif n == 1:
        b = []
        for i in table.split('<br>')[2:-1]:
            b.append(i.split(';'))
        n += n+2
    elif n == 4:
        c = []
        for i in table.split('<br>')[1:]:
            c.append(i.split(';'))

In [8]:
table_1 = pd.DataFrame(a[1:-1], columns=a[0])
table_2 = pd.DataFrame(b[1:-1], columns=b[0])
table_3 = pd.DataFrame(c[1:-1], columns=c[0])

In [9]:
table_1.to_csv('table_1.csv', index=False)
table_2.to_csv('table_2.csv', index=False)
table_3.to_csv('table_3.csv', index=False)